# 🧑‍🏫 Week 03 Class 02

**Querying SQLite with Python + SQL for Data Manipulation and Table Design**

🕒 Duration: 65 minutes

This class is split into three main parts:
	
1.	Python and SQLite Integration
2.	Data Manipulation (DML)
3.	Table Design (DDL)

---

## 🔹 Part 1: Data Manipulation Commands (20 min)

💡 Key SQL Syntax Examples (also used from Python):

**1.	INSERT:**

```sql
INSERT INTO students (name, grade) VALUES ('Bob', 90);
```

**2.	UPDATE:**

```sql
UPDATE students SET grade = 95 WHERE name = 'Bob';
```

**3.	DELETE:**

```sql
DELETE FROM students WHERE name = 'Bob';
```

**4. REPLACE (SQLite-specific):**

```sql
REPLACE INTO students (id, name, grade) VALUES (1, 'Alice', 88);
```

If row with id=1 exists, it’s deleted and replaced.

**5.	UPSERT with ON CONFLICT:**

```sql
INSERT INTO students (id, name, grade)
VALUES (1, 'Alice', 91)
ON CONFLICT(id) DO UPDATE SET grade = excluded.grade;
```

This command tries to insert a new student. If a student with the same `id = 1` already exists, instead of raising an error, it updates the existing row by setting its grade to 91.

**excluded** is a special keyword that refers to the values you tried to insert.

Only the column(s) explicitly listed in the UPDATE SET ... clause will be modified.

```{admonition} Talk to Your Neighbor
:class: tip
It the name changed? How would we update the name as well as the grade?
```

**6.	RETURNING:**

```sql
DELETE FROM students WHERE grade < 60 RETURNING *;
```


| Feature   | SQLite        | PostgreSQL     | MySQL                 |
|-----------|---------------|----------------|------------------------|
| REPLACE   | ✅ Yes        | ❌ No          | ✅ Yes                |
| UPSERT    | ✅ ON CONFLICT| ✅ ON CONFLICT | ✅ ON DUPLICATE KEY   |
| RETURNING | ✅ (3.35+)    | ✅             | ✅ (8.0.19+)          |

---

## 🔹 Part 2: Table Design and DDL (25 min)

### 🧱 Core DDL Commands

**1.	CREATE TABLE:**

```sql
CREATE TABLE courses (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    credits INTEGER
);
```

**2.	ALTER TABLE:**

```sql
ALTER TABLE courses ADD COLUMN description TEXT;
```

```{note}
SQLite only allows adding columns, not dropping or modifying them.
```

**3.	DROP TABLE:**

```sql
DROP TABLE IF EXISTS courses;
```

**4.	CREATE INDEX:**

```sql
CREATE INDEX idx_department ON courses(department);
```

**5.	DROP INDEX:**

```sql
DROP INDEX IF EXISTS idx_department;
```

**6.	CREATE VIEW:**

```sql
CREATE VIEW cs_courses AS
SELECT * FROM courses WHERE department = 'CS';
```

**7.	DROP VIEW:**

```sql
DROP VIEW IF EXISTS cs_courses;
```

SQLite views are read-only unless defined over simple queries.

---

## 🔹 Part 3: Querying SQLite from Python (20 min)

**🎯 Goal:**

Introduce students to running SQL commands directly from Python using the built-in sqlite3 library.

🔍 Key Concepts:

* How to connect to a SQLite database.
* How to execute SQL commands from Python.
* How to use placeholders to prevent SQL injection.
* How to retrieve query results.

### 🧑‍💻 Code Walkthrough

**1.	Connecting to a Database:**

In [1]:
import sqlite3
print(sqlite3.sqlite_version)

3.49.1


In [2]:
# Creating a connection to the database
# and a cursor object to execute SQL commands
# You can use ":memory:" to create a temporary database instead of a file, like this: 
# conn = sqlite3.connect(":memory:")
conn = sqlite3.connect("school.db")  # creates a file if it doesn’t exist
cursor = conn.cursor()

**2.	Creating a Table:**

In [3]:
# We drop the table to avoid duplicates. This won't be the case for all databases!
cursor.execute("""
DROP TABLE IF EXISTS students;
""")

# Creating a new table named 'students'
cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    grade INTEGER
)
""")

# Committing the changes to the database
conn.commit()

When you run SQL statements that modify the database (like `INSERT`, `UPDATE`, `DELETE`, `REPLACE`, `CREATE`, etc.), those changes are initially made in memory only as part of a transaction.

✅ conn.commit() makes those changes permanent in the .db file on disk.

**💡 Good Practices**

* Call `conn.commit()` after each logical unit of work (e.g., inserting multiple rows or after a form submission).
* For multiple steps, wrap them in a transaction using:

```python
with conn:
    cursor.execute(...)
    cursor.execute(...)
# Auto-commits on success, rolls back on error
```

**3.	Inserting Data with Parameters:**

In [4]:
cursor.execute("INSERT INTO students (name, grade) VALUES (?, ?)", ("Alice", 85))
cursor.execute("INSERT INTO students (name, grade) VALUES (?, ?)", ("Bob", 45))
cursor.execute("INSERT INTO students (name, grade) VALUES (?, ?)", ("Carl", 65))
cursor.execute("INSERT INTO students (name, grade) VALUES (?, ?)", ("Daniel", 95))
conn.commit()

**4.	Retrieving Data:**

In [5]:
cursor.execute("SELECT * FROM students")
for row in cursor.fetchall():
    print(row)

(1, 'Alice', 85)
(2, 'Bob', 45)
(3, 'Carl', 65)
(4, 'Daniel', 95)


**5.	Using RETURNING (SQLite 3.35+):**

In [6]:
# Returns all columns of the rows that were deleted.
cursor.execute("DELETE FROM students WHERE grade < 50 RETURNING *")
rows = cursor.fetchall()

print("Deleted rows:", rows)  # Should print: [(2, 'Bob', 45)]

Deleted rows: [(2, 'Bob', 45)]


## Activity 1

Create a New Table for Courses

Create a courses table with:

* id (PK)
* title
* credits

In [7]:
# your code in here

## Activity 2

Create a view high_achievers that shows students with grade ≥ 90.

```{note}
A view is a virtual table that stores the result of a SQL query.
It doesn’t store data itself — it just runs the underlying query whenever you access it.
```



In [8]:
# your code in here

## Activity 3

Query a view just like a table

In [9]:
# your code in here

## 🔥 Advanced View Example (Computed Column)

```sql
CREATE VIEW student_status AS
SELECT id, name, grade,
       CASE WHEN grade >= 60 THEN 'Pass' ELSE 'Fail' END AS status
FROM students;
```

To query it:

```sql
SELECT * FROM student_status;
```


In [10]:
# your code in here

In [11]:
# your code in here